<a href="https://colab.research.google.com/github/stelmanj/MusicAndLanguage/blob/master/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
install.packages("quanteda")

In [0]:
library(dplyr)
library(quanteda)
library(stringi)

Collecting data was a four step process and happened in python. The python code isn't posted, but I've summarized my steps below.
1. [This page on everynoise.com](http://everynoise.com/thesoundsofcountries.html) has links to 181 spotify playlists, each one specific to a different country. Through a combination of webscraping and requests to the [Spotify Web API](https://developer.spotify.com/documentation/web-api/), information was requested for all the songs on each of the 181 playlists to which this page contained a hyperlink. Those songs for which information was returned were made into a dataframe with columns for each piece of their Spotify metadata.
2. Through many requests to the [Musixmatch API](https://developer.musixmatch.com/), only a small portion of which were fruitful, excerpts of the songs' lyrics were collected and placed into a dataframe as well.
3. With the help of the [langdetect](https://github.com/Mimino666/langdetect) python library, the languages of a couple thousand of the lyric excerpts were able to be identified as one of several dozen languages.
4. Using $\verb|pandas|$, all these pieces were joined into one dataframe and exported as .csv file called [AFsongDF.csv](https://github.com/stelmanj/MusicAndLanguage/blob/master/AFsongDF.csv).


In [4]:
AFsongDF <- read.csv(
  "https://github.com/stelmanj/MusicAndLanguage/blob/master/AFsongDF.csv?raw=true",
  row.names = 1) 
# show 6 random rows
AFsongDF[sample(1:8286,6),]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,⋯,track_href,analysis_url,duration_ms,time_signature,title,artist,lyrics,lang,langs,sid
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
6432,0.380,0.589,9,-4.988,1,0.0300,0.4590,0.00e+00,0.132,0.364,⋯,https://api.spotify.com/v1/tracks/0cVfYSdy9g5wSM7XBkEEM9,https://api.spotify.com/v1/audio-analysis/0cVfYSdy9g5wSM7XBkEEM9,212360,4,La Más Fuerte,Ednita Nazario,"Como extraño verte y querer correr hacia tu boca Suplicarte que siempre durmieras justo al lado mío Como todo cambia, como cambia el clima en las ciudades Hoy entre los dos tan solamente habita el frío Tú quizás te niegas y me pides que luchemos Yo quizás llevo tanto intentando que no puedo Tu distancia me apago de a poco todo el fuego Y yo culpable en callarme No porque soy yo quien dice adiós No por eso yo te amaba menos No por eso soy la mala y vil de este cuento Solo de los dos soy la más fuerte Que prefiere ya no retenerte ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767)",es,[es:0.999995378379765],0cVfYSdy9g5wSM7XBkEEM9
3340,0.602,0.557,2,-7.835,1,0.0435,0.5330,2.11e-06,0.108,0.346,⋯,https://api.spotify.com/v1/tracks/5tu0LukKqDmwF6SJSeKaEI,https://api.spotify.com/v1/audio-analysis/5tu0LukKqDmwF6SJSeKaEI,186854,4,come home to,JED,no lyrics found,NA,NA,5tu0LukKqDmwF6SJSeKaEI
4823,0.652,0.627,11,-5.801,1,0.3470,0.0063,0.00e+00,0.090,0.324,⋯,https://api.spotify.com/v1/tracks/5Nm21isMaYPUlEaErQxeZQ,https://api.spotify.com/v1/audio-analysis/5Nm21isMaYPUlEaErQxeZQ,232516,4,Não Vou Te Deixar,Farao,"As vezes eu sinto que estou perdido Mais quando te vejo me reencontro Você é meu salo, me dás salário Es minha alegria no fim do mês Você me dignifica... Podem falar o que quiserem És o jindungo que pica, Picaii Podem me bater não desisto de ti Minha pequena eu nasci só pra ti Você é bem boa, bem boa, bem boa Juro mesmo weh, você mana você você mana você O que me fazes, você Nem por ouro vou te trocar você mana você você mana você O que me fazes, você Nem com macumba vou te deixere morê ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767)",pt,[pt:0.9999977513585101],5Nm21isMaYPUlEaErQxeZQ
1308,0.293,0.624,8,-4.262,1,0.0313,0.4980,7.56e-04,0.113,0.308,⋯,https://api.spotify.com/v1/tracks/7Mof2QVf5RIrZpr1XsDFYb,https://api.spotify.com/v1/audio-analysis/7Mof2QVf5RIrZpr1XsDFYb,250044,4,Bahagia,Eza Edmond,no lyrics found,NA,NA,7Mof2QVf5RIrZpr1XsDFYb
2910,0.640,0.878,5,-4.219,0,0.0363,0.0192,6.83e-06,0.364,0.739,⋯,https://api.spotify.com/v1/tracks/5mFIoGmRfz2Om9l6NljbV9,https://api.spotify.com/v1/audio-analysis/5mFIoGmRfz2Om9l6NljbV9,205393,4,Ne m'oublie pas,Carmelo,no lyrics found,NA,NA,5mFIoGmRfz2Om9l6NljbV9
7749,0.792,0.663,8,-10.003,1,0.0335,0.3570,0.00e+00,0.128,0.553,⋯,https://api.spotify.com/v1/tracks/6RTBd80nLFY1Nft7mJpFKx,https://api.spotify.com/v1/audio-analysis/6RTBd80nLFY1Nft7mJpFKx,194087,4,Sabeba,Cheb Bachir,no lyrics found,NA,NA,6RTBd80nLFY1Nft7mJpFKx


Some columns we don't need, like columns 5 and 16 among others. 

Those rows for songs either whose lyrics couldn't be found or whose lyrics' language couldn't be detected will have NaNs in some columns. That won't do.

In [0]:
dat <- na.exclude(AFsongDF[,c(-5,-16:-12,-23)])
dat$sid <- as.character(dat$sid) #The songid column is a character, not a factor, so make sure that is reflected.

Get the encoding types of each lyric excerpt.

In [0]:
stri_lang_enc = vapply(1:nrow(dat), function(x){
  stri_enc_detect(
    dat$lyrics[x], filter_angle_brackets = F
    )[[1]][c('Language','Encoding','Confidence')][1,] %>%
    unlist()},
    # Language: What language does R think this text is most likely written in (if it has an opinion)
    # Encoding: What type of encoding does R think is most likely right for this text
    # Confidence: We probably won't use, but how confident is R that this is the correct language diagnosis
  FUN.VALUE = c('Language','Encoding','Confidence')) %>% t() %>%
  as.data.frame()

# check it out
head(stri_lang_enc)

Make the text doc df for corpus analysis by sifting through all the ones with weird encoding types, and keeping only the ones we can handle. 

Also, remove sketchy inconsistencies: It's sometimes common for songs in other languages to have parts in English as well. That's just going to make things confusing. So hopefull this will help a little with removing those.

In [0]:
lyrics_df <- dat %>%
  # columns need to be renamed as doc_id and text so that later, the corpus function will know what to do with the data we give it
  select(doc_id = sid, text = lyrics, lang) %>%
  # throw the encoding columns into it )
  cbind(stri_lang_enc) %>%
  mutate_if(is.factor, as.character) %>%
  # get rid of non-ISO encodings
  filter(Encoding == "ISO-8859-1") %>%
  # if lang.detect said English but R said something else was more likely, get rid of it
  dplyr::filter(ifelse(lang != "en", T, ifelse(lang == Language, T, F)))

langs <- data.frame(lang = c('en', 'es', 'id', 'sw', 'nl'), stringsAsFactors = F)

# only use the languages in the 5 specified
lyrics_df <- lyrics_df %>% 
  inner_join(langs)

# clean up
rm(langs)

head(lyrics_df,2)

Joining, by = "lang"



,doc_id,text,lang,Language,Encoding,Confidence
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,5YPmo05u2Kd2HEGLFqaPXQ,Had lyom iss3id mbark ki wafitik a yamina ya taj lkhwdat khbark lmard libik hlkna w had lyom iss3id mbark ki wafitik a yamina ya taj lkhwdat khbark lmard libik hlkna mali tag 3lik nchofik we nssa9ssi f nass khyana mthawal galbi mndark kan lmola y9bl mna mali tag 3lik nchofik we nssa9ssi f nass khyana mthawal galbi mndark kan lmola y9bl mna hahahaa yamina w m3dm zinik al mglo3a hahahaa yamina w m3dm zinik a yamina blhkma chahrin nwdi hta dibt dwab lmlha kol lila w nhar nadi mayhlach nam lmha ra hobk ssfa mn jassdi w rabi 3ff w sabt raha nd3i lah ana njwjk ana chr3 w lh9 m3ana ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767),sw,hu,ISO-8859-2,0.18
2,5yqiverYlKEA0ly96HPq01,"Since 1992 there is a club which is making history. 7 years later, in 1999, it's still kicking: PONT AERI! When the stars begin to shine ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767)",en,en,ISO-8859-1,0.87


Take a smaller, stratified random sample. Limit to just five languages: English, Spanish, Indonesian, Dutch, and Swahili. Each strata is of 1 language and has 20 songs.

In [0]:
# take a smaller, evenly dispersed sample
set.seed(10)
## take an equal random sample from all five languages of interest
En_ids <- subset(x=lyrics_df,subset = lang == 'en')$doc_id
## n = 20 because there are exactly 22 songs in Dutch, the language of the 5 with the fewest songs
En_ids <- sample(En_ids,20)
eS_ids <- subset(x=lyrics_df,subset = lang == 'es')$doc_id
eS_ids <- sample(eS_ids,20)
Id_ids <- subset(x=lyrics_df,subset = lang == 'id')$doc_id
Id_ids <- sample(Id_ids,20)
Nl_ids <- subset(x=lyrics_df,subset = lang == 'nl')$doc_id
Nl_ids <- sample(Nl_ids,20)
sW_ids <- subset(x=lyrics_df,subset = lang == 'sw')$doc_id
sW_ids <- sample(sW_ids,20)

# replace a misclassified song
En_ids_all <- subset(x=lyrics_df,subset = lang == 'en')$doc_id
En_ids[which(En_ids == "720ZYTSr4vSqcFYq2CTJKN")] <-
  sample(En_ids_all[-which(En_ids_all %in% En_ids)],1)
rm(En_ids_all)
# replace another misclassified song
Id_ids_all <- subset(x=lyrics_df,subset = lang == 'id')$doc_id
Id_ids[which(Id_ids == "7mrxKs2fqNiKBE8zePEP2l")] <-
  sample(Id_ids_all[-which(Id_ids_all %in% Id_ids)],1)
rm(Id_ids_all)

# keep only the songs selected, and the columns we need
lyrics_df2 <- lyrics_df %>% subset(select = names(lyrics_df),
                   subset = doc_id %in% c(
                     En_ids, eS_ids, Id_ids, Nl_ids, sW_ids)) %>%
  select(doc_id, text, lang) # we don't need any of that encoding stuff anymore

head(lyrics_df2,2)


,doc_id,text,lang
,<chr>,<chr>,<chr>
69,5pz1Q9QFHWsUBZiJ73Jx3j,"Ze is niet altijd even vrolijk en dat ligt ook wel eens aan mij. En een beetje aan de weerman, maar die maakt eigenlijk niemand blij. En al zijn miezerige buien, daar heeft zij geen boodschap aan. Zij wil alle dagen zon, en als het moet eens een orkaan. Maar net als hem blijf ik proberen, elke dag een flauwe mop. Plots is daar dan toch die glimlach en dan klaart alles, dan klaart alles hier weer op. Want als ze lacht, breekt de hele hemel open, echt ik waan me in de tropen 't is echt machtig als ze lacht. Als ze lacht, baad ik uren in de zon ik wou dat ik dat voor haar kon wat zij voor mij doet als ze lacht. 'k Ben ook niet altijd even vrolijk, maar dat ligt echt wel niet aan mij. ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767)",nl
70,6eVj2ARRKQiy2Gb9Za7jnf,"Een lach, een groet, een blij gezicht Een vogel zwevend naar het licht. Oh het lijkt zo gewoon maar het is toch een wonder. Een kind dat lacht en naar je zwaait, een fietser die de hoek omdraait. Oh het lijkt zo gewoon maar het is toch een wonder. Het leven gaat zo snel voorbij, dat geld voor jou maar ook voor mij... Oh laat de zon in je hart Ze schijnt toch voor iedereen Geniet van het leven Want het duurt toch maar even. Oh aat de zon in je hart Ze schijnt toch voor iedereen Geniet van het leven ... ******* This Lyrics is NOT for Commercial use ******* (1409618588767)",nl


Create a corpus object out of the lyrics and calculate some metrics on these excerpts. 

In [0]:
# create a corpus of the lyrics
lyrics_corpus <-  corpus(lyrics_df2)

# Let's get some information about our corpus, shall we?

lyrics_sum <- summary(lyrics_corpus, tolower = T, n = 100) %>% 
  select(- Sentences) %>% # we don't care about sentences, song lyrics probably don't follow grammar protocols all that well
  mutate(Text = as.character(Text),
         TTR = Types/Tokens) %>% # add a column for Type-to-Token ratio
  select(-lang, lang) # and move lang to the end

head(lyrics_sum)

,Text,Types,Tokens,TTR,lang
,<chr>,<int>,<int>,<dbl>,<chr>
1,5YPmo05u2Kd2HEGLFqaPXQ,79,139,0.5683453,sw
2,5yqiverYlKEA0ly96HPq01,39,58,0.6724138,en
3,5UBIHAsEPBH2elLRRWb1SM,35,77,0.4545455,en
4,3XmpHN0jwCV1unk8ygs1Ku,46,101,0.4554455,es
5,5ISajXj2M1yvkrC0KsgbR5,36,65,0.5538462,es
6,4bfuNyCaeMEMCdxy4lJdVd,72,125,0.5760000,en


Let's bind that back together with the original data frame, take only the columns that we need, and make the data frame for Audio_Features_and_Lyric_Spread_Metrics

In [0]:
# dat has a lot of audio features that we want to use, and lyrics_sum has a metric we want to include
esinw_df <- dat %>% 
  dplyr::select(sid, danceability, energy, key, loudness, speechiness, 
                acousticness, valence, tempo, lang) %>%   # we only need some features
  mutate(lang = as.character(lang)) %>%   # remove the factor levels of lang
  mutate(lang = factor(lang)) %>%   # so that they can be reset to have only 5 levels
  inner_join(lyrics_sum %>%   # innerjoin with lyrics_sum, 
              mutate(sid = Text) %>%   # and rename the column "Text" to "sid"
              select(sid, Types, Tokens, TTR),   # and only keep the Text, Tokens, and TTR column 
            by = "sid")

# let's see what we got now
head(esinw_df,5)

,sid,danceability,energy,key,loudness,speechiness,acousticness,valence,tempo,lang,Types,Tokens,TTR
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>,<dbl>
1,5YPmo05u2Kd2HEGLFqaPXQ,0.751,0.813,1,-7.764,0.0388,0.51200,0.872,94.793,sw,79,139,0.5683453
2,5yqiverYlKEA0ly96HPq01,0.665,0.967,0,-7.776,0.0729,0.00518,0.389,157.979,en,39,58,0.6724138
3,5UBIHAsEPBH2elLRRWb1SM,0.674,0.394,9,-12.804,0.0409,0.48000,0.859,139.184,en,35,77,0.4545455
4,3XmpHN0jwCV1unk8ygs1Ku,0.778,0.813,5,-6.486,0.0304,0.37100,0.962,140.031,es,46,101,0.4554455
5,5ISajXj2M1yvkrC0KsgbR5,0.752,0.702,2,-5.024,0.0279,0.49800,0.853,100.017,es,36,65,0.5538462


In [0]:
# Let's save this to an .RData object so we can use it in Audio_Features_and_Lyric_Spread_Metrics
write.csv(esinw_df, file = "AFS_esinw.csv")

For A_Brief_Analysis_of_Lyric_Text_Metrics_in_5_Languages, we're still not quite done. First, we need to do some agreggating.

In [0]:
# get all the averages and totals we might want, grouping by language
lyrics_sum_avgd <- lyrics_sum %>% group_by(lang) %>% 
  summarise("Avg Tokens per Lyric Excerpt" = round(mean(Tokens)),
            "Avg Types per Lyric Excerpt" = round(mean(Types),2), 
            "Avg Type/Token Ratio (TTR)" = round(mean(TTR),3)) %>%  ungroup() %>%
  select(Language = lang, `Avg Tokens per Lyric Excerpt`, `Avg Types per Lyric Excerpt`, `Avg Type/Token Ratio (TTR)`) %>%
  as.data.frame()

# Tada!
lyrics_sum_avgd

Language,Avg Tokens per Lyric Excerpt,Avg Types per Lyric Excerpt,Avg Type/Token Ratio (TTR)
<chr>,<dbl>,<dbl>,<dbl>
en,136,72.20,0.550
es,83,41.00,0.505
id,56,35.00,0.625
nl,124,68.25,0.560
sw,139,79.00,0.568


Once more, but this time, rather than aggregating and averageing, we'll start by concatenating all excerpts of the same language together, and making a corpus out of that. Then will make a summary table for that hypothetical corpus so that later, we can compare it to the averaged one.

In [0]:
# concatenate songs of the same language together
lyrics_df_cat <- lyrics_df2 %>% group_by(lang) %>%
  summarise("text" = paste(text, collapse = " ")) %>%
  select(doc_id = lang, text)

# And create a new corpus object based on this
lyrics_corpus_cat <- corpus(lyrics_df_cat)

# fetch a summary of the corpus composition
lyrics_sum_cat <- summary(lyrics_corpus_cat, tolower = T) %>% 
  select(- Sentences) %>%
  mutate(Text = as.character(Text),
         TTR = round(Types/Tokens,3)) %>%
  rename(Language = Text)

lyrics_sum_cat

Language,Types,Tokens,TTR
<chr>,<int>,<int>,<dbl>
en,588,2543,0.231
es,564,2167,0.260
id,506,1528,0.331
nl,650,2569,0.253
sw,919,2229,0.412


In [0]:
# Let's save these guys so we can refer back to them later
write.csv(lyrics_sum_avgd, file = "BAT1_lsum_avgd.csv")
write.csv(lyrics_sum_cat, file = "BAT2_lsum_cat.csv")